In [ ]:
database_name = ''

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
session.sql(f"""
    CREATE OR REPLACE TABLE {database_name}.PUBLIC.DDL_EXPORT(
        OBJECT_NAME STRING,
        DDL STRING
    );
""").collect()

In [ ]:
table_df = session.sql(f"""
    SELECT TABLE_CATALOG||'.'||TABLE_SCHEMA||'.'||TABLE_NAME AS OBJECT_NAME
    FROM {database_name}.INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA NOT IN ('INFORMATION_SCHEMA','PUBLIC','SNOWFLAKE_TRACKER');
""").to_pandas()

print(table_df['OBJECT_NAME'])

In [ ]:
import pandas as pd

# Collect DDLs
object_names = []
ddls = []

for table_name in table_df['OBJECT_NAME']:
    try:
        ddl_result = session.sql(
            "SELECT GET_DDL('TABLE', ?) AS DDL",
            params=[table_name]
        ).collect()
        
        object_names.append(table_name)
        ddls.append(ddl_result[0]['DDL'])
        
    except Exception as e:
        print(f"✗ {table_name}: {e}")

# Create DataFrame
result_df = pd.DataFrame({
    'OBJECT_NAME': object_names,
    'DDL': ddls
})

# Write to local CSV first
local_file = '/tmp/ddl_export.csv'
result_df.to_csv(local_file, index=False, quoting=1)  # quoting=1 quotes all fields

session.sql("""
    CREATE OR REPLACE STAGE DDL_EXPORT_STAGE
        FILE_FORMAT = (
            TYPE = 'CSV'
            FIELD_DELIMITER = ','
            SKIP_HEADER = 1
            FIELD_OPTIONALLY_ENCLOSED_BY = '"'
            NULL_IF = ('NULL', 'null', '')
            EMPTY_FIELD_AS_NULL = TRUE
            COMPRESSION = 'AUTO'
        );
""").collect()

session.file.put(local_file, "@DDL_EXPORT_STAGE", auto_compress=False, overwrite=True)